# Equity

In [88]:
import pandas as pd
import warnings
from nse import nse_self

warnings.filterwarnings("ignore")
nse = nse_self()

today = "22-08-2025"
symbols = ["HDFCBANK", "IDBI", "HINDUNILVR", "EQUITASBNK", "CIPLA"]

def stock_highs_and_gaps(symbol):
    # 1Y, 1M, 1W historical data
    data_1y = nse.get_nse_stock_by_duration(symbol=symbol, period="1Y")[::-1]
    data_1m = nse.get_nse_stock_by_duration(symbol=symbol, period="1M")[::-1]
    data_1w = nse.get_nse_stock_by_duration(symbol=symbol, period="1W")[::-1]

    closes = data_1y["ClosePrice"].replace({",": ""}, regex=True).astype(float)
    highs_1y = data_1y["HighPrice"].replace({",": ""}, regex=True).astype(float)
    highs_1m = data_1m["HighPrice"].replace({",": ""}, regex=True).astype(float)
    highs_1w = data_1w["HighPrice"].replace({",": ""}, regex=True).astype(float)

    today_close = closes.iloc[0]
    high_52 = highs_1y.max()
    high_1m = highs_1m.max()
    high_1w = highs_1w.max()

    gap_52 = round(((high_52 - today_close) / high_52) * 100, 2)
    gap_1m = round(((high_1m - today_close) / high_1m) * 100, 2)
    gap_1w = round(((high_1w - today_close) / high_1w) * 100, 2)

    return {
        "Symbol": symbol,
        "52W High": high_52,
        "Today Close": today_close,
        "Gap_52W": f"-{gap_52}%",
        "Gap_1M": f"-{gap_1m}%",
        "Gap_1W": f"-{gap_1w}%",
    }


rows = [stock_highs_and_gaps(sym) for sym in symbols]
df = pd.DataFrame(rows)

# highlight gap columns
def highlight_gaps(val):
    try:
        num = float(val.strip('%').replace('-', ''))  # remove % and minus sign
        if num > 10:
            return "font-weight: bold; color: red;"
    except:
        return ""
    return "font-weight: bold;"

styled_df = (
    df.style
      .applymap(highlight_gaps, subset=["Gap_52W", "Gap_1M", "Gap_1W"])
      .format({
          "52W High": "{:.2f}",
          "Today Close": "{:.2f}",
          "Gap_52W": "{}",
          "Gap_1M": "{}",
          "Gap_1W": "{}"
      })
)
styled_df


,Symbol,52W High,Today Close,Gap_52W,Gap_1M,Gap_1W
0,HDFCBANK,2037.70,1964.60,-3.59%,-3.59%,-3.06%
1,IDBI,106.32,95.00,-10.65%,-4.04%,-4.04%
2,HINDUNILVR,3035.00,2629.90,-13.35%,-3.58%,-1.87%
3,EQUITASBNK,86.90,54.88,-36.85%,-13.22%,-1.98%
4,CIPLA,1702.05,1592.80,-6.42%,-0.57%,-0.57%


# Crypto

In [90]:
import cryptocompare as cp
import pandas as pd

current = cp.get_price(['BTC', 'ETH'], currency='USD')
btc = current['BTC']['USD']
eth = current['ETH']['USD']
# print(btc, eth)

btc_prices = cp.get_historical_price_day('BTC',
                                currency='USD',
                                limit=29)
btc_highs = [day['high'] for day in btc_prices]
btc_high = max(btc_highs)

eth_prices = cp.get_historical_price_day('ETH',
                                currency='USD',
                                limit=29)
eth_highs = [day['high'] for day in eth_prices]
eth_high = max(eth_highs)
# print(btc_high, eth_high)

btc_gap = ((btc_high-btc)/btc)*100
eth_gap = ((eth_high-eth)/eth)*100
# print(btc_gap, eth_gap)

data = [
    ["BTC", btc_high, btc, btc_gap],
    ["ETH", eth_high, eth, eth_gap]
]
df = pd.DataFrame(data, columns=["Symbol", "1M High", "Current", "Gap"])

def highlight_gap(val):
    if val > 5:
        return 'color: red; font-weight: bold;'
    return 'font-weight: bold;'

styled_df = df.style.applymap(highlight_gap, subset=['Gap']) \
                   .format({
                       "1M High": "{:.2f}",
                       "Current": "{:.2f}",
                       'Gap': '-{:.2f}%'
                    })
styled_df

,Symbol,1M High,Current,Gap
0,BTC,124532.70,114383.35,-8.87%
1,ETH,4902.24,4895.07,-0.15%
